In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
df_spacex = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv", index_col = 0)
df_spacex.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
max_payload = df_spacex['Payload Mass (kg)'].max()
min_payload = df_spacex['Payload Mass (kg)'].min()
print(f'Max payload - {max_payload:.2f}, min_payload - {min_payload:.2f}')

Max payload - 9600.00, min_payload - 0.00


In [4]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'All Sites'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='All Sites',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):    
    if entered_site == 'All Sites':
        fig = px.pie(df_spacex, 
                     values='class', 
                     names='Launch Site', 
                     title='Total Success Launches By Site')        
    else:
        df_filt = df_spacex[df_spacex['Launch Site'] == entered_site]
        df_filt = df_filt[['class','Flight Number']].groupby('class').count().reset_index()        
        fig = px.pie(df_filt, 
                     values='Flight Number', 
                     names='class', 
                     title=f'Total Launches for site {entered_site}')        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value"))
def get_scatter_chart(entered_site, payload_range):
    if entered_site == 'All Sites':
        df_filt = df_spacex[(df_spacex['Payload Mass (kg)'] >= int(payload_range[0])) & (df_spacex['Payload Mass (kg)'] <= int(payload_range[1]))]
        fig = px.scatter(df_filt, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f'All sites - payload mass between {int(payload_range[0]):8} kg and {int(payload_range[1]):8} kg')
    else:
        df_filt = df_spacex[(df_spacex['Launch Site'] == entered_site) & (df_spacex['Payload Mass (kg)'] >= int(payload_range[0])) & (df_spacex['Payload Mass (kg)'] <= int(payload_range[1]))                           ]
        fig = px.scatter(df_filt, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f'Site {entered_site} - payload mass between {int(payload_range[0]):8} kg and {int(payload_range[1]):8} kg')
    
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode = 'inline')